## Querying historical MTHexapod data

In [ ]:
import pandas as pd
from string import Template
from lsst_efd_client import EfdClient

## July 2022

USDF has a database with the EFD historical data until August 2022. So you can query July 2022 data using:

In [ ]:
client = EfdClient("usdf_efd", db_name="efd-s2022-08-10")

In [ ]:
# Query MTHexapod heartbeats

start = '2022-07-01'
end = '2022-07-31'
query = f'''SELECT count("heartbeat") AS "count_heartbeat" FROM "lsst.sal.MTHexapod.logevent_heartbeat" WHERE time > '{start}' AND time < '{end}' GROUP BY time(1h) '''
heartbeats = await client.influx_client.query(query)


In [ ]:
heartbeats.plot(y="count_heartbeat")

In [ ]:
# Query MTHexapod actuators data (salIndex=1 means means Camera Hexapod)

start = '2022-07-01'
end = '2022-07-31'
query = f'''SELECT "calibrated0", "calibrated1", "calibrated2", "calibrated3", "calibrated4", "calibrated5" FROM  "lsst.sal.MTHexapod.actuators" WHERE salIndex=1 AND time > '{start}' AND time < '{end}' '''
actuators = await client.influx_client.query(query)


In [ ]:
actuators.plot(y="calibrated0")

In [ ]:
actuators[actuators['calibrated0']>0]

## August 2022
August 2022 data is restored at the Base EFD on a shard basis. That  means we have to query individual shards (a week worth of data) and concatenate the partial results.

For that we need to map the shards and their database:

In [ ]:
august2022 = [
    {"db_name": "efd-s1071", "start": "2022-08-08", "end": "2022-08-15"},
    {"db_name": "efd-s1078", "start": "2022-08-15", "end": "2022-08-22"},
    {"db_name": "efd-s1088", "start": "2022-08-22", "end": "2022-08-29"}
]

We also need this helper function:

In [ ]:
async def query_shards(period, query):

    df = pd.Series()
    
    for shard in period:

        client = EfdClient("summit_efd_copy", db_name=shard["db_name"])
        q = query.substitute(start=shard["start"], end=shard["end"])
        print(f"DEBUG: {q}")
        partial = await client.influx_client.query(q)
        
        if type(partial) is not dict:
            df = pd.concat([df, partial])
        
    return df

And a template query that has the `start` and `end` variables. 

In [ ]:
# Query MTHexapod heartbeats

query = Template('''SELECT count("heartbeat") AS "count_heartbeat" FROM "lsst.sal.MTHexapod.logevent_heartbeat" WHERE time > '$start' AND time < '$end' GROUP BY time(1h) ''')
heartbeats = await query_shards(august2022, query)

In [ ]:
heartbeats.plot(y="count_heartbeat")

In [ ]:
# Query MTHexapod actuators data (salIndex=1 means means Camera Hexapod)

query = Template('''SELECT "calibrated0", "calibrated1", "calibrated2", "calibrated3", "calibrated4", "calibrate5" FROM "lsst.sal.MTHexapod.actuators" WHERE salIndex=1 AND time > '$start' AND time < '$end' ''')
actuators = await query_shards(august2022, query)

In [ ]:
actuators.plot(y="calibrated0")

### September 2022
September 2022 data is also restored at the Base EFD on a shard basis. 

In [ ]:
september2022 = [
    {"db_name": "efd-s597", "start": "2022-08-29", "end": "2022-09-05"},
    {"db_name": "efd-s602", "start": "2022-09-05", "end": "2022-09-12"},
    {"db_name": "efd-s921", "start": "2022-09-12", "end": "2022-09-19"},
    {"db_name": "efd-s932", "start": "2022-09-19", "end": "2022-09-26"},
    {"db_name": "efd-s943", "start": "2022-09-26", "end": "2022-10-03"}
]

In [ ]:
# Query MTHexapod heartbeats

query = Template('''SELECT count("heartbeat") AS "count_heartbeat" FROM "lsst.sal.MTHexapod.logevent_heartbeat" WHERE time > '$start' AND time < '$end' GROUP BY time(1h) ''')

heartbeats = await query_shards(september2022, query)

In [ ]:
heartbeats.plot(y="count_heartbeat")

In [ ]:
# Query MTHexapod actuators data (salIndex=1 means means Camera Hexapod)

query = Template('''SELECT "calibrated0", "calibrated1", "calibrated2", "calibrated3", "calibrated4", "calibrated5" FROM "lsst.sal.MTHexapod.actuators" WHERE salIndex=1 AND time > '$start' AND time < '$end' ''')
actuators = await query_shards(september2022, query)

In [ ]:
actuators.plot(y="calibrated0")

In [ ]:
actuators[actuators['calibrated0']>0]

### October 2022
October 2022 data is also restored at the Base EFD on a shard basis. 

In [ ]:
october2022 = [
    {"db_name": "efd-s987", "start": "2022-10-24", "end": "2022-10-31"},
    {"db_name": "efd-s976", "start": "2022-10-17", "end": "2022-10-24"},
    {"db_name": "efd-s965", "start": "2022-10-10", "end": "2022-10-17"},
    {"db_name": "efd-s954", "start": "2022-10-03", "end": "2022-10-10"}
]

In [ ]:
# Query MTHexapod heartbeats

query = Template('''SELECT count("heartbeat") AS "count_heartbeat" FROM "lsst.sal.MTHexapod.logevent_heartbeat" WHERE time > '$start' AND time < '$end' GROUP BY time(1h) ''')

heartbeats = await query_shards(october2022, query)

In [ ]:
heartbeats.plot(y="count_heartbeat")

In [ ]:
# Query MTHexapod actuators data (salIndex=1 means means Camera Hexapod)

query = Template('''SELECT "calibrated0", "calibrated1", "calibrated2", "calibrated3", "calibrated4", "calibrated5" FROM "lsst.sal.MTHexapod.actuators" WHERE salIndex=1 AND time > '$start' AND time < '$end' ''')
actuators = await query_shards(october2022, query)

In [ ]:
actuators.plot(y="calibrated0")

In [ ]:
actuators[actuators['calibrated0']>0]